In [1]:
from tqdm import tqdm
import pandas as pd
import os
import pickle
from time import sleep
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open('query_dict/id_querry_clean.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)
with open('title_dict/norm_title_data.pickle', 'rb') as file:
    title_data = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
global_corp = []
for key in tqdm(title_data):
    global_corp.append(title_data[key])

100%|██████████| 582167/582167 [00:00<00:00, 2387286.02it/s]


In [5]:
tfidf_vectorizer_1 = TfidfVectorizer(ngram_range=(1,1))
tfidf_vectorizer_1.fit(global_corp)
global_corp = []
title_classic_feats = dict()
title_classic_feats['title_tfidf_vectorizer_1'] = []

In [6]:
def calc_cos_sim(vect, title, query):
    x = vect.transform([title])
    q = vect.transform([query])
    return cosine_similarity(x,q).flatten()[0]

In [7]:
q_ids = all_groups['QueryId'].values
t_ids = all_groups['DocumentId'].values
for i in tqdm(range(0, 606050)):
    cur_q = id_querry_clean[q_ids[i]]
    cur_t = title_data[t_ids[i]]
    title_classic_feats['title_tfidf_vectorizer_1'].append(calc_cos_sim(tfidf_vectorizer_1, cur_t, cur_q))

100%|██████████| 606050/606050 [2:06:41<00:00, 79.73it/s]  


In [8]:
with open('title_tfidf_vectorizer_1.pickle', 'wb') as file:
    pickle.dump(title_classic_feats, file)